# 第4章: 言語解析

問題30から問題35までは、以下の文章`text`（太宰治の『走れメロス』の冒頭部分）に対して、言語解析を実施せよ。問題36から問題39までは、国家を説明した文書群（日本語版ウィキペディア記事から抽出したテキスト群）をコーパスとして、言語解析を実施せよ。

In [14]:
text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

In [11]:
!apt install mecab libmecab-dev mecab-ipadic-utf8 -y
!pip install mecab-python3
import MeCab

# mecabrc の場所を明示（Colabは /etc/mecabrc にある）
mecab = MeCab.Tagger("-r /etc/mecabrc -Ochasen")
print(mecab.parse("私は昨日、図書館へ行って本を読みました。"))

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libmecab-dev is already the newest version (0.996-14build9).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-3).
mecab is already the newest version (0.996-14build9).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
私	ワタシ	私	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
昨日	キノウ	昨日	名詞-副詞可能		
、	、	、	記号-読点		
図書館	トショカン	図書館	名詞-一般		
へ	ヘ	へ	助詞-格助詞-一般		
行っ	イッ	行く	動詞-自立	五段・カ行促音便	連用タ接続
て	テ	て	助詞-接続助詞		
本	ホン	本	名詞-一般		
を	ヲ	を	助詞-格助詞-一般		
読み	ヨミ	読む	動詞-自立	五段・マ行	連用形
まし	マシ	ます	助動詞	特殊・マス	連用形
た	タ	た	助動詞	特殊・タ	基本形
。	。	。	記号-句点		
EOS



## 30. 動詞
文章`text`に含まれる動詞をすべて表示せよ。

In [16]:
import MeCab

mecab = MeCab.Tagger("-r /etc/mecabrc")

node = mecab.parseToNode(text)
while node:
    # 品詞情報は feature に入っている
    features = node.feature.split(",")
    if features[0] == "動詞":  # 品詞が動詞
        print(node.surface)    # 単語の表層形を出力
    node = node.next

し
除か
なら
し
わから
吹き
遊ん
暮し
来


## 31. 動詞の原型
文章`text`に含まれる動詞と、その原型をすべて表示せよ。

In [19]:
node = mecab.parseToNode(text)
while node:
    # 品詞情報は feature に入っている
    features = node.feature.split(",")
    if features[0] == "動詞":  # 品詞が動詞
        print(f'動詞:{node.surface},原型:{features[6]}')    # 単語の表層形を出力
    node = node.next

動詞:し,原型:する
動詞:除か,原型:除く
動詞:なら,原型:なる
動詞:し,原型:する
動詞:わから,原型:わかる
動詞:吹き,原型:吹く
動詞:遊ん,原型:遊ぶ
動詞:暮し,原型:暮す
動詞:来,原型:来る


## 32. 「AのB」
文章`text`において、2つの名詞が「の」で連結されている名詞句をすべて抽出せよ。

In [23]:
node = mecab.parseToNode(text)
results=[]

while node:
  features = node.feature.split(",")
  # 品詞情報が名詞か
  if features[0] == "名詞":  # 品詞が動詞
    first = node.surface
    next1=node.next
    if next1:
      features2 = next1.feature.split(",")
      #次がの
      if features2[0] == "助詞" and features2[6] == "の":
        next2=next1.next
        if next2:
          features3 = next2.feature.split(",")
          #次が名詞
          if features3[0] == "名詞":
            second = next2.surface
            results.append(f'{first}の{second}')

  node = node.next
for phrase in results:
    print(phrase)

暴虐の王
村の牧人


## 33. 係り受け解析

文章`text`に係り受け解析を適用し、係り元と係り先のトークン（形態素や文節などの単位）をタブ区切り形式ですべて抽出せよ。

In [26]:
!pip install -U ginza
!python -m spacy download ja_ginza


✘ No compatible package found for 'ja_ginza' (spaCy v3.8.7)



In [31]:
!pip install ginza ja-ginza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.9 MB/s eta 0:00:00


In [32]:
import spacy

nlp = spacy.load("ja_ginza")

doc = nlp(text)

for token in doc:
    if token.dep_ != "ROOT":  # 係り先があるものだけ
        print(f"{token.head.text}\t{token.text}")

メロス	

激怒	メロス
メロス	は
激怒	し
激怒	た
激怒	。
除か	必ず
必ず	、
暴虐	かの
暴虐	邪智
王	暴虐
暴虐	の
除か	王
王	を
決意	除か
除か	なけれ
なけれ	ば
なけれ	なら
なけれ	ぬ
除か	と
決意	し
決意	た
決意	。
メロス	

わから	メロス
メロス	に
メロス	は
わから	政治
政治	が
わから	ぬ
わから	。
メロス	

牧人	メロス
メロス	は
メロス	、
牧人	村
村	の
牧人	で
で	ある
牧人	。
笛	

吹き	笛
笛	を
暮し	吹き
吹き	、
遊ん	羊
羊	と
暮し	遊ん
遊ん	で
暮し	て
て	来
暮し	た
暮し	。
邪悪	


	けれど

	も
敏感	邪悪
邪悪	に
に	対し
に	ては
邪悪	、
倍	人
倍	一
敏感	倍
倍	に
敏感	で
で	あっ
敏感	た
敏感	。


## 34. 主述の関係
文章`text`において、「メロス」が主語であるときの述語を抽出せよ。

In [33]:

for token in doc:
    # tokenが動詞などの述語かつ、主語が「メロス」のものを探す
    if token.pos_ in ("VERB", "AUX"):  # 動詞・助動詞
        # その述語の主語を探す
        for child in token.children:
            if child.dep_ == "nsubj" and child.text == "メロス":
                print(token.text)

激怒


## 35. 係り受け木
「メロスは激怒した。」の係り受け木を可視化せよ。

## 36. 単語の出現頻度

問題36から39までは、Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](/data/jawiki-country.json.gz)をコーパスと見なし、統計的な分析を行う。

* 1行に1記事の情報がJSON形式で格納される
* 各行には記事名が"title"キーに、記事本文が"text"キーの辞書オブジェクトに格納され、そのオブジェクトがJSON形式で書き出される
* ファイル全体はgzipで圧縮される

まず、第3章の処理内容を参考に、Wikipedia記事からマークアップを除去し、各記事のテキストを抽出せよ。そして、コーパスにおける単語（形態素）の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

## 37. 名詞の出現頻度
コーパスにおける名詞の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

## 38. TF・IDF
日本に関する記事における名詞のTF・IDFスコアを求め、TF・IDFスコア上位20語とそのTF, IDF, TF・IDFを表示せよ。

## 39. Zipfの法則
コーパスにおける単語の出現頻度順位を横軸、その出現頻度を縦軸として、両対数グラフをプロットせよ。